In [9]:
# '''
# Take in raw csv file path and batch size, return DataLoader object
# '''

import csv
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
# from sklearn.model_selection import KFold

def load_data(file_path, batch_size):
    inputs = []
    labels = []

    # Read the CSV file and load data into lists
    with open(file_path, "r") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # skip header row
        for row in reader:
            labels.append(float(row[0]))
            inputs.append([float(val) for val in row[1:]])

    # Convert inputs and labels to tensors
    inputs_tensor = torch.tensor(inputs, dtype=torch.float32)
    labels_tensor = torch.tensor(labels, dtype=torch.float32)

    # Normalize the inputs using Z-score normalization
    means = inputs_tensor.mean(dim=0)  # Mean for each feature
    stds = inputs_tensor.std(dim=0)    # Standard deviation for each feature
    inputs_normalized = (inputs_tensor - means) / stds  # Z-score normalization

    # Combine inputs and labels into a TensorDataset
    dataset = TensorDataset(inputs_normalized, labels_tensor)

    total_size = len(dataset)
    train_size = int(0.8 * total_size)
    val_size = int(0.1 * total_size)
    test_size = total_size - train_size - val_size  # To handle rounding

    # Split the dataset
    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)  # For reproducibility
    )

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

In [10]:
import torch.nn as nn

# Define the neural network model
class DiabetesNN(nn.Module):
    def __init__(self, input_dim=21, hidden_size1=64, hidden_size2=32, activation_fn="relu"):
        super(DiabetesNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.output = nn.Linear(32, 1)
        if activation_fn == "sigmoid":
            self.activation = torch.sigmoid
        elif activation_fn == "tanh":
            self.activation = torch.tanh
        elif activation_fn == "relu":
            self.activation = torch.relu
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.layer1(x))
        x = self.activation(self.layer2(x))
        x = self.output(x)
        return x

In [ ]:
import torch.optim as optim

def train_model(model, train_loader, num_epochs, learning_rate):
    # Use BCEWithLogitsLoss for binary classification
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        epoch_loss = 0
        for data, target in train_loader:
            target = target.float()  # Convert target to float
            optimizer.zero_grad()  # Reset gradients
            output = model(data).squeeze(1)  # Forward pass, squeeze for shape match
            loss = criterion(output, target)  # Calculate BCEWithLogitsLoss

            loss.backward()  # Backpropagation
            optimizer.step()  # Update parameters based on gradients

            epoch_loss += loss.item()  # Accumulate loss

        # Print epoch loss
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")


In [ ]:
'''
Train model with train set. 
'''

file_path = "diabetes_binary_health_indicators_BRFSS2015.csv"

# Hyperparams
hidden_size = 7
batch_size = 8
activation_fn = "relu"
num_epochs = 100
learning_rate = 1

train_loader, val_loader, test_loader = load_data(file_path, batch_size)
model = DiabetesNN(hidden_size, activation_fn)

train_model(model, train_loader, num_epochs, learning_rate)

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            # between the probabilities for 0 and 1, pick higher
            preds = torch.argmax(output, dim=1)
            all_preds.extend(preds.numpy())
            all_targets.extend(target.numpy())

    all_preds = torch.tensor(all_preds).squeeze()
    all_targets = torch.tensor(all_targets)
    
    correct = (all_preds == all_targets).sum().item()
    total = len(all_targets)
    accuracy = correct / total
    
    print(f"Accuracy: {accuracy:.4f}")
    return accuracy

In [ ]:
evaluate_model(DiabetesNN, val_loader)